# Canine Colonocyte Butyrate Metabolism: Model Caninization Workflow

**Objective**: Convert human colonocyte butyrate metabolism model (Human-GEM) to canine ortholog-based model.

**Approach**:
1. Load Human-GEM SBML model
2. Filter butyrate pathway reactions
3. Map human → dog gene orthologs
4. Substitute GPR (Gene-Protein-Reaction) associations
5. Validate model integrity
6. Apply physiological bounds
7. Run FBA/pFBA analysis
8. Visualize and export results

---

## Block 1: Setup & Imports

In [ ]:
# Core libraries
import cobra
from cobra.flux_analysis import pfba, flux_variability_analysis
import pandas as pd
import numpy as np
import re

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"COBRApy version: {cobra.__version__}")
print(f"Pandas version: {pd.__version__}")
print("\n✓ All imports successful")

## Block 2: Load Human-GEM Model

**Model source**: Human-GEM (Recon3D) - download from [GitHub](https://github.com/SysBioChalmers/Human-GEM)

**Expected**: ~13,400 reactions, ~4,100 genes

In [ ]:
# Load Human-GEM model (adjust path as needed)
# Download from: https://github.com/SysBioChalmers/Human-GEM/releases
MODEL_PATH = "../data/Human-GEM.xml"  # Update this path

try:
    model_human = cobra.io.read_sbml_model(MODEL_PATH)
    print(f"✓ Model loaded: {model_human.id}")
    print(f"  Reactions: {len(model_human.reactions)}")
    print(f"  Metabolites: {len(model_human.metabolites)}")
    print(f"  Genes: {len(model_human.genes)}")
except FileNotFoundError:
    print(f"⚠ Model file not found at {MODEL_PATH}")
    print("  Download Human-GEM.xml from GitHub and place in data/ folder")
    model_human = None

## Block 3: Filter Butyrate Pathway Reactions

**Target subsystems**:
- SCFA transport (lumen → cytosol)
- Butyrate activation (→ butyryl-CoA)
- β-oxidation (C4 pathway)
- Electron transfer (ETFA/B/DH)
- OXPHOS (Complexes I-V)

In [ ]:
# Search butyrate-related reactions
if model_human:
    # Keywords for butyrate metabolism
    keywords = ['but', 'butyr', 'butyrate', 'c4', 'scfa', 'mct1', 'smct', 'slc16a1', 'slc5a8']
    
    but_reactions = []
    for rxn in model_human.reactions:
        rxn_str = (rxn.id + " " + rxn.name).lower()
        if any(kw in rxn_str for kw in keywords):
            but_reactions.append(rxn)
    
    print(f"Found {len(but_reactions)} butyrate-related reactions")
    print("\nSample reactions:")
    for rxn in but_reactions[:10]:
        print(f"  {rxn.id:30s} {rxn.name}")
    
    # Extract unique genes from these reactions
    but_genes = set()
    for rxn in but_reactions:
        but_genes.update(rxn.genes)
    
    print(f"\n✓ {len(but_genes)} genes associated with butyrate reactions")
else:
    print("⚠ Skipping - model not loaded")

### Create Butyrate Sub-Model

Extract core pathway + essential reactions:
- Butyrate reactions (filtered above)
- OXPHOS (ATP synthase, respiratory chain)
- Exchange reactions (EX_but, EX_o2, EX_co2, etc.)
- ATPM (ATP maintenance)

In [ ]:
if model_human:
    # Core genes to include (from ortholog mapping file)
    core_genes = [
        'SLC16A1',  # MCT1
        'SLC5A8', 'SLC5A12',  # SMCT1/2
        'ACSM2A', 'ACSM2B', 'ACSS3',  # Activation
        'ACADS',  # β-oxidation
        'ECHS1', 'HADHA', 'HADHB', 'ACAT1',  # β-oxidation
        'ETFA', 'ETFB', 'ETFDH'  # Electron transfer
    ]
    
    # Find reactions associated with core genes
    core_reactions = set()
    for gene_id in core_genes:
        gene = model_human.genes.get_by_id(gene_id) if gene_id in model_human.genes else None
        if gene:
            core_reactions.update(gene.reactions)
            print(f"  {gene_id:12s} → {len(gene.reactions)} reactions")
    
    print(f"\n✓ Total core reactions: {len(core_reactions)}")
    
    # Add essential subsystems
    essential_keywords = ['oxphos', 'respiratory', 'atp synthase', 'atpm', 'complex']
    for rxn in model_human.reactions:
        rxn_str = (rxn.id + " " + rxn.name + " " + (rxn.subsystem or "")).lower()
        if any(kw in rxn_str for kw in essential_keywords):
            core_reactions.add(rxn)
    
    # Add exchange reactions
    for rxn in model_human.exchanges:
        if any(met in rxn.id.lower() for met in ['but', 'o2', 'co2', 'h2o', 'h_', 'pi']):
            core_reactions.add(rxn)
    
    print(f"✓ After adding OXPHOS + exchanges: {len(core_reactions)} reactions")
else:
    print("⚠ Skipping - model not loaded")

## Block 4: Load Ortholog Mappings

Load human → dog gene mappings from curated Excel file.

In [ ]:
# Load ortholog mapping file
ORTHOLOG_FILE = "../data/02_human_dog_orthologs.xlsx"

try:
    df_orthologs = pd.read_excel(ORTHOLOG_FILE)
    print(f"✓ Loaded {len(df_orthologs)} ortholog mappings")
    print("\nColumns:", list(df_orthologs.columns))
    print("\nSample mappings:")
    display(df_orthologs.head())
    
    # Create mapping dictionary: human_gene → dog_gene
    ortholog_map = dict(zip(
        df_orthologs['Gene umano'],  # Adjust column name if different
        df_orthologs['Ortologo canino']
    ))
    
    print(f"\n✓ Created mapping for {len(ortholog_map)} genes")
    
except FileNotFoundError:
    print(f"⚠ File not found: {ORTHOLOG_FILE}")
    ortholog_map = {}
except Exception as e:
    print(f"⚠ Error loading file: {e}")
    print("  Check column names and file format")
    ortholog_map = {}

## Block 5: Substitute GPRs (Human → Dog)

Replace human gene IDs with canine orthologs in Gene-Protein-Reaction rules.

**GPR Logic**:
- `AND`: protein complex (all genes required)
- `OR`: isozymes (any gene sufficient)

In [ ]:
if model_human and ortholog_map:
    # Create canine model (copy)
    model_canine = model_human.copy()
    model_canine.id = "CanineColon_Butyrate"
    
    # Track substitutions
    substituted = 0
    not_found = set()
    
    for rxn in model_canine.reactions:
        gpr_original = rxn.gene_reaction_rule
        if not gpr_original:
            continue
        
        # Extract human gene IDs from GPR
        gene_ids = [g.id for g in rxn.genes]
        
        # Substitute genes
        gpr_canine = gpr_original
        for human_gene in gene_ids:
            if human_gene in ortholog_map:
                dog_gene = ortholog_map[human_gene]
                # Replace gene ID in GPR string
                gpr_canine = re.sub(
                    r'\b' + re.escape(human_gene) + r'\b',
                    dog_gene,
                    gpr_canine
                )
                substituted += 1
            else:
                not_found.add(human_gene)
        
        # Update GPR
        if gpr_canine != gpr_original:
            rxn.gene_reaction_rule = gpr_canine
    
    print(f"✓ Substituted {substituted} gene occurrences")
    print(f"  Genes without mapping: {len(not_found)}")
    if not_found:
        print("\nUnmapped genes (sample):")
        for gene in list(not_found)[:10]:
            print(f"  - {gene}")
    
    print(f"\n✓ Canine model created: {model_canine.id}")
    print(f"  Reactions: {len(model_canine.reactions)}")
    print(f"  Genes: {len(model_canine.genes)}")
else:
    print("⚠ Skipping - prerequisites not met")
    model_canine = None

## Block 6: Validate Model Integrity

**Checks**:
1. GPR integrity (no broken parentheses)
2. Orphan reactions (reactions without genes in critical pathways)
3. Mass/charge balance
4. Core pathway completeness

In [ ]:
if model_canine:
    print("=== Model Validation ===")
    
    # 1. Check for reactions without GPR (orphans)
    orphan_reactions = [r for r in model_canine.reactions if not r.genes]
    print(f"\n1. Orphan reactions (no GPR): {len(orphan_reactions)}")
    
    # Check core pathway orphans
    core_orphans = [r for r in core_reactions if not r.genes]
    if core_orphans:
        print(f"   ⚠ {len(core_orphans)} orphans in CORE pathway:")
        for rxn in core_orphans[:5]:
            print(f"     - {rxn.id}: {rxn.name}")
    else:
        print("   ✓ No orphans in core butyrate pathway")
    
    # 2. Check GPR syntax (balanced parentheses)
    broken_gpr = []
    for rxn in model_canine.reactions:
        gpr = rxn.gene_reaction_rule
        if gpr and gpr.count('(') != gpr.count(')'):
            broken_gpr.append(rxn.id)
    
    print(f"\n2. GPR syntax errors: {len(broken_gpr)}")
    if broken_gpr:
        print(f"   ⚠ Broken GPRs: {broken_gpr[:5]}")
    else:
        print("   ✓ All GPRs syntactically valid")
    
    # 3. Check core genes presence
    print("\n3. Core gene verification:")
    for gene_id in core_genes:
        dog_gene = ortholog_map.get(gene_id, "NOT_MAPPED")
        present = dog_gene in model_canine.genes
        status = "✓" if present else "✗"
        print(f"   {status} {gene_id:12s} → {dog_gene:20s}")
    
    print("\n✓ Validation complete")
else:
    print("⚠ Skipping - model not available")

## Block 7: Apply Physiological Bounds & Run FBA

Apply "healthy dog" baseline constraints and test metabolic flux.

In [ ]:
# Load physiological bounds
BOUNDS_FILE = "../data/01_physiological_bounds.xlsx"

try:
    df_bounds = pd.read_excel(BOUNDS_FILE)
    print("✓ Loaded physiological bounds")
    display(df_bounds)
except FileNotFoundError:
    print(f"⚠ File not found: {BOUNDS_FILE}")
    df_bounds = None

In [ ]:
if model_canine and df_bounds is not None:
    print("=== Applying Bounds ===")
    
    # Apply bounds from dataframe
    for _, row in df_bounds.iterrows():
        rxn_id = row['Reaction_ID']
        lb = row['Lower Bound (mmol/gDW/h)']
        ub = row['Upper Bound (mmol/gDW/h)']
        
        if rxn_id in model_canine.reactions:
            rxn = model_canine.reactions.get_by_id(rxn_id)
            rxn.lower_bound = float(lb)
            rxn.upper_bound = float(ub)
            print(f"  {rxn_id:15s} [{lb:7.1f}, {ub:7.1f}]")
        else:
            print(f"  ⚠ Reaction not found: {rxn_id}")
    
    # Set objective: ATP maintenance (ATPM)
    if 'ATPM' in model_canine.reactions:
        model_canine.objective = 'ATPM'
        print("\n✓ Objective set: ATPM")
    else:
        # Search for alternative
        atpm_candidates = [r.id for r in model_canine.reactions if 'atpm' in r.id.lower()]
        if atpm_candidates:
            model_canine.objective = atpm_candidates[0]
            print(f"\n✓ Objective set: {atpm_candidates[0]}")
        else:
            print("\n⚠ ATPM reaction not found")
    
    print("\n=== Running FBA ===")
    solution = model_canine.optimize()
    
    print(f"Status: {solution.status}")
    print(f"Objective value (ATPM): {solution.objective_value:.6f} mmol/gDW/h")
    
    # Display key fluxes
    print("\nKey fluxes:")
    key_rxns = ['EX_but_lum', 'EX_ac_lum', 'EX_pro_lum', 'EX_o2_bld', 'ATPM']
    for rxn_id in key_rxns:
        if rxn_id in model_canine.reactions:
            flux = solution.fluxes.get(rxn_id, 0.0)
            print(f"  {rxn_id:15s} {flux:10.6f}")
    
    # Run pFBA (parsimonious FBA)
    print("\n=== Running pFBA ===")
    pfba_solution = pfba(model_canine)
    print(f"pFBA objective: {pfba_solution.objective_value:.6f}")
    print(f"Total flux: {pfba_solution.fluxes.abs().sum():.2f}")
    
else:
    print("⚠ Skipping - prerequisites not met")

## Block 8: Results Visualization & Export

Visualize flux distributions and export canine model.

In [ ]:
if model_canine and solution.status == 'optimal':
    # Plot key fluxes
    key_rxns = ['EX_but_lum', 'EX_ac_lum', 'EX_pro_lum', 'EX_o2_bld']
    fluxes = [solution.fluxes.get(r, 0) for r in key_rxns]
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.barh(key_rxns, fluxes, color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12'])
    ax.set_xlabel('Flux (mmol/gDW/h)')
    ax.set_title('Key Metabolite Exchange Fluxes - Canine Colon Model')
    ax.axvline(0, color='black', linewidth=0.8)
    plt.tight_layout()
    plt.show()
    
    # Export canine model
    output_path = "../data/CanineColon_Butyrate.xml"
    cobra.io.write_sbml_model(model_canine, output_path)
    print(f"\n✓ Canine model exported: {output_path}")
    
    # Export flux results
    flux_df = pd.DataFrame({
        'Reaction': solution.fluxes.index,
        'Flux_FBA': solution.fluxes.values,
        'Flux_pFBA': pfba_solution.fluxes.values
    })
    flux_df = flux_df[flux_df['Flux_FBA'].abs() > 1e-6]  # Filter near-zero
    flux_df.to_csv("../data/canine_flux_results.csv", index=False)
    print(f"✓ Flux results exported: ../data/canine_flux_results.csv")
    
    print("\n" + "="*60)
    print("✓ CANINIZATION WORKFLOW COMPLETE")
    print("="*60)
else:
    print("⚠ Cannot visualize - optimization failed or model unavailable")

---

## Next Steps

1. **Validate with experimental data**: Compare predicted fluxes with measured SCFA concentrations
2. **Sensitivity analysis**: Test robustness to parameter variations
3. **Dysbiosis scenarios**: Simulate reduced butyrate availability
4. **Expand model**: Add glycolysis, TCA cycle, amino acid metabolism
5. **Microbiota integration**: Couple with microbial community models

---

**Version**: 1.0  
**Date**: 2025-01-12  
**Author**: [Your name]